In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
import numpy as np
import plotly.graph_objs as go
import pandas
import os
import psutil #to get the memory used
import os


In [2]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(device)

cuda


In [10]:
combined_sequences = []

leave = []  # List to store indices of files not considered
####change foolder location
dirs = os.listdir('NMP')
c = 0
for d in dirs:
    count = len(list(os.listdir(os.path.join('NMP', d))))
    print(count)
    for i in range(count):
        c += 1
        # Load the .npz file
        ####change foolder location before NMP
        data = np.load(f'NMP/{d}/{d}_{i}.npz')
        
        # Extract relevant data
        position_data = data['position']  # Assuming 'position' is a key in the .npz file
        
        # Extract x, y, z coordinates
        x = position_data[:, 0][:100]  # Get the first 100 x coordinates
        y = position_data[:, 1][:100]  # Get the first 100 y coordinates
        z = position_data[:, 2][:100]  # Get the first 100 z coordinates
        
        # Check if the sequence is long enough
        if len(x) == 100:
            combined_sequence = np.vstack((x, y, z)).T  # Stack x, y, z and transpose to get shape (100, 3)
            combined_sequences.append(combined_sequence)
             ####change foolder location before NMP
            print(f'NMP/trimmed_Bamboo/trimmed_Bamboo_{i}.npz - Loaded successfully')
        else:
             ####change foolder location before NMP
            leave.append(f'NMP/{d}/{d}_{i}.npz')
            print(f'NMP/trimmed_Bamboo/trimmed_Bamboo_{i}.npz - left')
        
        print("---------------------------")


    # Convert list to NumPy array
    combined_array = np.array(combined_sequences)

    # Convert NumPy array to PyTorch tensor
    combined_tensor = torch.tensor(combined_array, dtype=torch.float32)


    # Now combined_tensor can be used for training your LSTM model
print(combined_tensor.shape)  # Should print torch.Size([163, 100, 3])

print(c)

168
NMP/trimmed_Bamboo/trimmed_Bamboo_0.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_1.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_2.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_3.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_4.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_5.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_6.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_7.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_8.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_9.npz - Loaded successfully
---------------------------
NMP/trimmed_Bamboo/trimmed_Bamboo_10.npz - Loaded successfully
---------------------------
NMP/t

In [11]:
combined_tensor = combined_tensor.to(device)

In [12]:

test_data = np.load(f'NMP/trimmed_Bamboo/trimmed_Bamboo_{47}.npz')

position_data = test_data['position'][:100]  # Limit to the first 100 samples if necessary

mid_point = position_data.shape[0] // 2

# Prepare the input and target tensors
test_input = torch.tensor(position_data[:mid_point], dtype=torch.float32).unsqueeze(0)  # First half
test_target = torch.tensor(position_data[mid_point:], dtype=torch.float32).unsqueeze(0)  # Second half

# Transfer tensors to the appropriate device (GPU or CPU)
test_input_4 = test_input.to(device)
test_target_4 = test_target.to(device)


In [13]:
position_data = torch.tensor(position_data)
position_data = position_data.to(device)

In [14]:
combined_tensor

tensor([[[ 1.0006,  1.2619,  1.8557],
         [ 0.9993,  1.3053,  1.8195],
         [ 0.9999,  1.3470,  1.7853],
         ...,
         [ 0.8238,  2.0998, -1.2494],
         [ 0.8214,  2.0753, -1.2787],
         [ 0.8192,  2.0507, -1.3079]],

        [[ 1.0979,  1.2421,  1.8938],
         [ 1.0934,  1.2864,  1.8607],
         [ 1.0895,  1.3295,  1.8273],
         ...,
         [ 0.8766,  2.2128, -1.1789],
         [ 0.8738,  2.1914, -1.2095],
         [ 0.8710,  2.1696, -1.2403]],

        [[ 0.9545,  0.7337, -2.0168],
         [ 0.9555,  0.7594, -1.9725],
         [ 0.9570,  0.7893, -1.9274],
         ...,
         [ 1.1758,  1.4907,  1.7906],
         [ 1.1789,  1.4662,  1.8246],
         [ 1.1822,  1.4405,  1.8593]],

        ...,

        [[ 1.3530,  0.6545,  3.4689],
         [ 1.3512,  0.6912,  3.4029],
         [ 1.3490,  0.7340,  3.3395],
         ...,
         [ 0.9526,  2.3103, -1.0961],
         [ 0.9489,  2.2920, -1.1348],
         [ 0.9454,  2.2731, -1.1734]],

        [[

In [9]:
'''res_x = pandas.DataFrame(dtype=float) # append x 
res_y = pandas.DataFrame(dtype=float) # append y
res_z = pandas.DataFrame(dtype=float) #append z
leave = [] #append files which are not considered
for i in range(168):

    data = np.load(f'NMP/trimmed_Bamboo/trimmed_Bamboo_{i}.npz') #loading the file based on i value
    df = pandas.DataFrame.from_dict({item: data[item] for item in data.files}, orient='index') #converting into dataframe
    df.drop(['quaternion','frame_num', 'time_step'], inplace=True) #dropping unecessary columns
    #print(df)
    #concat
    info = df.T # Transpose the dataframe
    # Extract x, y, z positions
    info['x'] = info['position'].apply(lambda pos: pos[0])
    info['y'] = info['position'].apply(lambda pos: pos[1])
    info['z'] = info['position'].apply(lambda pos: pos[2])
    #info = info.drop(['position'], inplace=True, axis = 1)
    info.drop(['position'], inplace=True, axis=1)
    info = info.T
    # Transpose to keep it as a row
    x = pandas.DataFrame(info.loc['x'].iloc[:100]).T  
    y = pandas.DataFrame(info.loc['y'].iloc[:100]).T  
    z = pandas.DataFrame(info.loc['z'].iloc[:100]).T 
    
    if not df.shape[1] < 100: #if greater than 100 concat will happen
        info = info.iloc[:, :100]
        res_x = pandas.concat([res_x, x]) #default axis = 0
        res_y = pandas.concat([res_y, y]) #default axis = 0
        res_z = pandas.concat([res_z, z]) #default axis = 0
        print(f'NMP/trimmed_Bamboo/trimmed_Bamboo_{i}.npz')
        print(df)
    elif df.shape[1] < 100:
        leave.append(i)
    print("---------------------------")

#print('total files concatinated count: ', 168 - len(leave))
    '''

'res_x = pandas.DataFrame(dtype=float) # append x \nres_y = pandas.DataFrame(dtype=float) # append y\nres_z = pandas.DataFrame(dtype=float) #append z\nleave = [] #append files which are not considered\nfor i in range(168):\n\n    data = np.load(f\'NMP/trimmed_Bamboo/trimmed_Bamboo_{i}.npz\') #loading the file based on i value\n    df = pandas.DataFrame.from_dict({item: data[item] for item in data.files}, orient=\'index\') #converting into dataframe\n    df.drop([\'quaternion\',\'frame_num\', \'time_step\'], inplace=True) #dropping unecessary columns\n    #print(df)\n    #concat\n    info = df.T # Transpose the dataframe\n    # Extract x, y, z positions\n    info[\'x\'] = info[\'position\'].apply(lambda pos: pos[0])\n    info[\'y\'] = info[\'position\'].apply(lambda pos: pos[1])\n    info[\'z\'] = info[\'position\'].apply(lambda pos: pos[2])\n    #info = info.drop([\'position\'], inplace=True, axis = 1)\n    info.drop([\'position\'], inplace=True, axis=1)\n    info = info.T\n    # Trans

In [10]:
'''# may use for column wise input
def forward(self, x, future=0): #updated forwad
        outputs = []
        n_samples = x.size(0)  # Batch size
        seq_len = x.size(1)  # Number of time steps
        n_features = x.size(2)  # Number of features (should be 3 for x, y, z)

        h_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32)
        c_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) 
        h_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) 
        c_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) 

        # Iterate over each time step
        for t in range(seq_len):
            input = x[:, t, :]  # Extract the t-th time step across all sequences; shape is [batch_size, 3]
            h_t, c_t = self.lstm1(input, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)  # output will have shape [batch_size, 3]
            outputs.append(output)

        # Predict future values if required
        for i in range(future):
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs.append(output)

        outputs = torch.stack(outputs, dim=1)  # Stack the outputs along the sequence dimension
        return outputs'''

'# may use for column wise input\ndef forward(self, x, future=0): #updated forwad\n        outputs = []\n        n_samples = x.size(0)  # Batch size\n        seq_len = x.size(1)  # Number of time steps\n        n_features = x.size(2)  # Number of features (should be 3 for x, y, z)\n\n        h_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32)\n        c_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) \n        h_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) \n        c_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32) \n\n        # Iterate over each time step\n        for t in range(seq_len):\n            input = x[:, t, :]  # Extract the t-th time step across all sequences; shape is [batch_size, 3]\n            h_t, c_t = self.lstm1(input, (h_t, c_t))\n            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))\n            output = self.linear(h_t2)  # output will have shape [batch_size, 3]\n            outputs.append(output)\n\n

In [18]:
class lstmpred(nn.Module):
    def __init__(self, n_hidden=51):
        super(lstmpred, self).__init__()
        self.n_hidden = n_hidden
        self.lstm1 = nn.LSTMCell(3, self.n_hidden)
        self.lstm2 = nn.LSTMCell(self.n_hidden, self.n_hidden)
        self.lstm3 = nn.LSTMCell(self.n_hidden, self.n_hidden)  # lstm3
        self.lstm4 = nn.LSTMCell(self.n_hidden, self.n_hidden)  # lstm4
        self.lstm5 = nn.LSTMCell(self.n_hidden, self.n_hidden)  # lstm5 (new layer)
        self.lstm6 = nn.LSTMCell(self.n_hidden, self.n_hidden)  # lstm6 (new layer)
        self.linear = nn.Linear(self.n_hidden, 3)

    def forward(self, x, future=0):
        outputs = []
        n_samples = x.size(0)  # Batch size

        # Initialize hidden and cell states for all LSTM layers
        h_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        c_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        h_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        c_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        h_t3 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        c_t3 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        h_t4 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        c_t4 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)
        #h_t5 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)  # For lstm5
        #c_t5 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)  # For lstm5
        #h_t6 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)  # For lstm6
        #c_t6 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32).to(device)  # For lstm6

        for input in x.split(1, dim=1):  # Input shape [batch_size, 1, 3]
            input = input.squeeze(1)  # Shape [batch_size, 3]
            h_t, c_t = self.lstm1(input, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            h_t3, c_t3 = self.lstm3(h_t2, (h_t3, c_t3))
            h_t4, c_t4 = self.lstm4(h_t3, (h_t4, c_t4))
            #h_t5, c_t5 = self.lstm5(h_t4, (h_t5, c_t5))  # Add lstm5 layer processing
            #h_t6, c_t6 = self.lstm6(h_t5, (h_t6, c_t6))  # Add lstm6 layer processing
            output = self.linear(h_t4)  # Output from the last LSTM layer (lstm6)
            outputs.append(output)

        for i in range(future):
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            h_t3, c_t3 = self.lstm3(h_t2, (h_t3, c_t3))
            h_t4, c_t4 = self.lstm4(h_t3, (h_t4, c_t4))
            #h_t5, c_t5 = self.lstm5(h_t4, (h_t5, c_t5))  # Add lstm5 layer processing
            #h_t6, c_t6 = self.lstm6(h_t5, (h_t6, c_t6))  # Add lstm6 layer processing
            output = self.linear(h_t4)
            outputs.append(output)

        outputs = torch.stack(outputs, dim=1)  # Shape [batch_size, sequence_length, 3]
        return outputs


In [19]:
train_input = combined_tensor[3:, :-1]   # leave the last time step for training input
train_target = combined_tensor[3:, 1:]   # training target is shifted by one step
test_input = combined_tensor[:3, :-1]    # first 3 sequences for testing
test_target = combined_tensor[:3, 1:]    # testing target is also shifted by one step

In [ ]:
###this is for  overall trajectory optimisation
# Initialize the model
model = lstmpred()  # No .cuda() call here
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.LBFGS(model.parameters(), lr=0.8)
n_steps = 30  # Number of steps to train the model

for i in range(n_steps):
    print("Step:", i + 1)

    def closure():
        optimizer.zero_grad()
        out = model(train_input)
        loss = criterion(out, train_target)
        print('Training loss:', loss.item())
        loss.backward()
        return loss

    optimizer.step(closure)

    # Testing phase
    with torch.no_grad():
        future = 100  # Predict 100 future steps beyond the test sequence
        pred = model(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print('Test loss:', loss.item())
        y_pred = pred.cpu().detach().numpy()

In [17]:
# Model initialization
model = lstmpred().to(device)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.LBFGS(model.parameters(), lr=0.8)

# Training for 1100 sequences
n_steps = 30  # Number of optimization steps

for i in range(n_steps):
    print("Step:", i + 1)

    # Closure function for LBFGS optimizer
    def closure():
        optimizer.zero_grad()
        total_loss = 0.0

        for start in range(99):  # Iterate over possible starting points
            train_input = combined_tensor[:, :start + 1, :]  # Use start+1 columns as input
            train_target = combined_tensor[:, start + 1:, :]  # Predict the remaining columns

            # Check tensor sizes for debugging
            assert train_input.size(1) <= combined_tensor.size(1), f"Input size {train_input.size()} exceeds tensor limits"
            assert train_target.size(1) + start + 1 == combined_tensor.size(1), f"Target size {train_target.size()} mismatch"

            out = model(train_input, future=train_target.size(1))
            loss = criterion(out[:, -train_target.size(1):], train_target)
            total_loss += loss

        total_loss.backward()
        print(f'Training loss at step {i+1}: {total_loss.item()}')

        # Check memory consumption
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0] / 2.**30  # Convert from bytes to GB
        print(f"Memory Usage: {memoryUse:.2f} GB")
        return total_loss

    # Optimizer step
    optimizer.step(closure)

    # Testing phase
    with torch.no_grad():
        future = 99  # Predict remaining steps for the last test sequence
        pred = model(test_input, future=future)
        loss = criterion(pred[:, -future:], test_target)
        print(f'Test loss at step {i+1}: {loss.item()}')

        # Check and print shape of the prediction for debugging
        print(f'Prediction shape: {pred.shape}')
        y_pred = pred.cpu().detach().numpy()


Step: 1
Training loss: 237.10369873046875
Memory Usage: 1.33 GB
Training loss: 229.4971160888672
Memory Usage: 1.69 GB
Training loss: 67.62924194335938
Memory Usage: 1.61 GB
Training loss: 19.801706314086914
Memory Usage: 1.64 GB
Training loss: 17.361705780029297
Memory Usage: 1.64 GB
Training loss: 16.849802017211914
Memory Usage: 1.64 GB
Training loss: 16.838558197021484
Memory Usage: 1.65 GB
Training loss: 16.83818817138672
Memory Usage: 1.64 GB
Training loss: 16.838153839111328
Memory Usage: 1.64 GB
Training loss: 16.838123321533203
Memory Usage: 1.64 GB
Training loss: 16.838041305541992
Memory Usage: 1.65 GB
Training loss: 16.837852478027344
Memory Usage: 1.64 GB
Training loss: 16.83739471435547
Memory Usage: 1.64 GB
Training loss: 16.836275100708008
Memory Usage: 1.64 GB
Training loss: 16.833101272583008
Memory Usage: 1.64 GB
Training loss: 16.828184127807617
Memory Usage: 1.64 GB
Training loss: 16.81668472290039
Memory Usage: 1.65 GB
Training loss: 16.79482650756836
Memory Usage

In [21]:
position_data[:,1]

tensor([1.1956, 1.2291, 1.2633, 1.2960, 1.3281, 1.3592, 1.3894, 1.4187, 1.4471,
        1.4751, 1.5007, 1.5276, 1.5528, 1.5783, 1.6021, 1.6251, 1.6474, 1.6692,
        1.6906, 1.7119, 1.7319, 1.7516, 1.7707, 1.7895, 1.8074, 1.8250, 1.8428,
        1.8596, 1.8758, 1.8917, 1.9069, 1.9216, 1.9359, 1.9495, 1.9626, 1.9749,
        1.9866, 1.9974, 2.0076, 2.0170, 2.0256, 2.0330, 2.0402, 2.0464, 2.0518,
        2.0563, 2.0599, 2.0629, 2.0650, 2.0664, 2.0672, 2.0672, 2.0661, 2.0641,
        2.0619, 2.0586, 2.0545, 2.0496, 2.0441, 2.0384, 2.0313, 2.0240, 2.0157,
        2.0070, 1.9975, 1.9872, 1.9763, 1.9647, 1.9523, 1.9399, 1.9265, 1.9125,
        1.8981, 1.8829, 1.8667, 1.8508, 1.8339, 1.8160, 1.7973, 1.7794, 1.7612,
        1.7420, 1.7222, 1.7016, 1.6807, 1.6598, 1.6372, 1.6155, 1.5922, 1.5688,
        1.5449, 1.5205, 1.4953, 1.4696, 1.4435, 1.4165, 1.3891, 1.3609, 1.3320,
        1.3025], device='cuda:0')

In [22]:
test_target_4.size(1)

50

In [23]:

model.eval()
with torch.no_grad():
    # Predict the second half of the sequence based on the first half
    future_steps = test_target_4.size(1)  # Number of future steps equals the length of the second half
    prediction = model(test_input_4, future=future_steps)

    # Extract the predicted second half
    predicted_output = prediction[:, -future_steps:]  # Only take the predicted future steps

    # Move the tensors to CPU and convert to NumPy arrays
    prediction = prediction.cpu().detach().numpy()
    predicted_output = predicted_output.cpu().detach().numpy()

In [24]:
position_data = position_data.cpu().numpy()

In [25]:
prediction.squeeze(0)

array([[ 1.1848636 ,  1.0191939 ,  2.940102  ],
       [ 1.2697089 ,  1.2225137 ,  2.889707  ],
       [ 1.2372739 ,  1.3135138 ,  2.855091  ],
       [ 1.2398351 ,  1.3286476 ,  2.8057575 ],
       [ 1.2401571 ,  1.334722  ,  2.7461574 ],
       [ 1.2351289 ,  1.3368726 ,  2.6841278 ],
       [ 1.2274365 ,  1.3429632 ,  2.6239448 ],
       [ 1.2193787 ,  1.3554964 ,  2.5679584 ],
       [ 1.2120488 ,  1.3741605 ,  2.5158656 ],
       [ 1.2057416 ,  1.3977075 ,  2.466902  ],
       [ 1.2003715 ,  1.4245932 ,  2.4205441 ],
       [ 1.1954937 ,  1.4541702 ,  2.3735118 ],
       [ 1.1907816 ,  1.4854934 ,  2.3257108 ],
       [ 1.18587   ,  1.5181482 ,  2.2755127 ],
       [ 1.1806054 ,  1.5513844 ,  2.2239866 ],
       [ 1.1748096 ,  1.5848532 ,  2.1704745 ],
       [ 1.1682961 ,  1.6183867 ,  2.114248  ],
       [ 1.160918  ,  1.6518741 ,  2.0548973 ],
       [ 1.1526476 ,  1.6851506 ,  1.9929295 ],
       [ 1.1436625 ,  1.7178998 ,  1.9302447 ],
       [ 1.1338725 ,  1.7503009 ,  1.864

In [26]:
prediction = np.squeeze(prediction, axis=0)

In [27]:
prediction.shape

(100, 3)

In [28]:
# Extract x, y, z coordinates from the predictions
predicted_x = prediction[:, 0]
predicted_y = prediction[:, 1]
predicted_z = prediction[:, 2]


actual_x = position_data[:, 0]
actual_y = position_data[:, 1]
actual_z = position_data[:, 2]

# Create the 3D line plot for the actual data (first half)
trace_actual = go.Scatter3d(
    x=actual_x,
    y=actual_y,
    z=actual_z,
    mode='lines+markers',  # Both lines and markers
    marker=dict(size=5, color='blue'),  # Marker size and color for actual data
    line=dict(color='blue', width=2),  # Line color and width for actual data
    name='Actual Data (First Half)'
)

# Create the 3D line plot for the predicted data (second half)
trace_predicted = go.Scatter3d(
    x=predicted_x,
    y=predicted_y,
    z=predicted_z,
    mode='lines+markers',  # Both lines and markers
    marker=dict(size=5, color='red'),  # Marker size and color for predicted data
    line=dict(color='red', width=2),  # Line color and width for predicted data
    name='Predicted Data (Second Half)'
)

# Create layout
layout = go.Layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title="Interactive 3D Trajectory: Actual (First Half) and Predicted (Second Half)"
)

# Create the figure with both traces and display it
fig = go.Figure(data=[trace_actual, trace_predicted], layout=layout)
fig.show()